# Inversion Intensity

Tracks left/right radiation points and X-point, including brightness (intensity) for D-IIID TV images

In [ ]:
import numpy as np
from scipy.io import readsav
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import cv2
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from sklearn.linear_model import LinearRegression

### Data Loading

In [ ]:
def _load_data(filename):
    dat = readsav(filename)
    emission = dat['emission_structure']
    return emission[0]

def _find_index(arr,val):
    return np.argmin(abs(arr-val))

In [ ]:
filepath = Path('tv_images/l-mode')
filename = 'emission_structure_pu_cam240perp_190113'
fullfile = filename + '.sav'
[inverted,radii,elevation,frames,times,vid_frames,vid_times,vid] = _load_data(filepath / fullfile)

### Static Image With Corner Detection

Inverted: R,Z Coordinate Array
Radii/Elevation are redundant across times. Can just use ones from t=0.

In [ ]:

fid = 50
tid = _find_index(vid_times,times[fid]) #find frame id for camera image with t=times[fid]
find_x = inverted[fid]
img = inverted[fid].copy()
gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
useHarrisDetector = False # False uses Shi-Tomasi Corner Detector
corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=useHarrisDetector))
x = radii[0][corners[:,0,0]]
y = elevation[0][corners[:,0,1]]
print(np.column_stack((x,y)))
plt.pcolormesh(radii[fid],elevation[fid],img,shading='auto', cmap='gray')
plt.scatter(x,y,color='red')
plt.title(f"Inverted with Radiation Points, Frame {fid}")
plt.show()

### Dynamic Point Tracking

In [ ]:
def get_avg_value(array):
    return np.sqrt(np.mean(np.square(array)))

# converts from natural units to indices
def convert_center(radii, elevation, value):
    rad_idx = (np.abs(radii - value[0])).argmin()
    elev_idx = (np.abs(elevation - value[1])).argmin()
    return rad_idx, elev_idx

# distance from test point to line created by point_1 and point_2
def get_dist_line(point_1, point_2, test_point):
    top = np.abs(point_2[0]-point_1[0])*(point_2[1]-test_point[1])-(point_1[0]-test_point[0])*(point_2[1]-point_1[1])
    bottom = np.sqrt((point_2[0]-point_1[0])**2+(point_2[1]-point_1[1])**2)
    return top / bottom

In [ ]:
# boundary array for frame around center point
def get_bounds(centers, dist, im_size):
    
    bounds = np.array([centers - dist, centers + dist])
    
    bounds[bounds < 0] = 0 # set negative bounds to 0
    
    new_vals = bounds.copy()
    new_vals[new_vals > im_size[0]] = im_size[0]
    bounds[:,:,0] = new_vals[:,:,0] # set x bounds within x frame
    
    new_vals = bounds.copy()
    new_vals[new_vals > im_size[1]] = im_size[1]
    bounds[:,:,1] = new_vals[:,:,1] # set y bounds within y frame
    
    return bounds

# array of corners using Shi-Tomasi Corner Detector
def get_corners(img):
    
    gray = (255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
    corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.05,5, useHarrisDetector=False))
    x = corners[:,0,0]
    y = corners[:,0,1]
    
    return np.column_stack((x,y))

# array of corner pixel frame intensities
def get_corner_values(img, corners, dist, im_size):
    
    bounds = get_bounds(corners, dist, im_size)
    
    avg_values = []
    
    for i in range(len(corners)):
        temp_frame = img[bounds[0,i,1]:bounds[1,i,1],bounds[0,i,0]:bounds[1,i,0]]
        avg_values.append(get_avg_value(temp_frame))
    
    return avg_values

Finding and merging the most likely radiation points
1. X-point is absolute
1. Merge radiation points that are close together
1. Provided that there also exists a corner that is close enough to the previous line formed by the x-point and radiation point, then replace radiation point with corner

In [ ]:
# merges points that are close together and sets new radiation points to possible corners if they meet threshold criteria
def merge_points(centers_old, centers_new, corners, avg_values, avg_values_corners, merge_threshold, distance_threshold):
    
    right_dist = []
    left_dist = []
    centers_new_update = centers_new.copy()
    avg_values_update = avg_values.copy()
    
    for i in range(len(corners)):
        right_dist.append(get_dist_line(centers_old[0],centers_old[1],corners[i]))
        left_dist.append(get_dist_line(centers_old[0],centers_old[2],corners[i]))

    l_x_dist = np.sqrt(np.sum(np.square(centers_new[0]-centers_new[1])))
    r_x_dist = np.sqrt(np.sum(np.square(centers_new[0]-centers_new[2])))
    
    if (l_x_dist < merge_threshold) and (np.min(left_dist)) < distance_threshold:
        index = np.where(right_dist == np.min(left_dist))[0][0]
        centers_new_update[1] = corners[index]
        avg_values_update[1] = avg_values_corners[index]
    
    if (r_x_dist < merge_threshold) and (np.min(right_dist) < distance_threshold):
        index = np.where(right_dist == np.min(right_dist))[0][0]
        centers_new_update[2] = corners[index]
        avg_values_update[2] = avg_values_corners[index]
    
    return centers_new_update, avg_values_update

# array of values for x point and 2 radiation points
def get_center_values(img, centers, dist, im_size, merge_threshold, distance_threshold):
    
    bounds = get_bounds(centers, dist, im_size)
    
    x_frame = img[bounds[0,0,1]:bounds[1,0,1],bounds[0,0,0]:bounds[1,0,0]]
    l_frame = img[bounds[0,1,1]:bounds[1,1,1],bounds[0,1,0]:bounds[1,1,0]]
    r_frame = img[bounds[0,2,1]:bounds[1,2,1],bounds[0,2,0]:bounds[1,2,0]]
    
    local_x_max = np.unravel_index(np.argmax(x_frame), x_frame.shape)
    local_l_max = np.unravel_index(np.argmax(l_frame), l_frame.shape)
    local_r_max = np.unravel_index(np.argmax(r_frame), r_frame.shape)
    
    global_x_max = np.flip(np.ravel(local_x_max)) + [bounds[0,0,0], bounds[0,0,1]]
    global_l_max = np.flip(np.ravel(local_l_max)) + [bounds[0,1,0], bounds[0,1,1]]
    global_r_max = np.flip(np.ravel(local_r_max)) + [bounds[0,2,0], bounds[0,2,1]]
    
    centers_update = np.array([global_x_max, global_l_max, global_r_max])
    avg_values = np.array([get_avg_value(x_frame), get_avg_value(l_frame), get_avg_value(r_frame)])
    
    corners = get_corners(img)
    avg_values_corners = get_corner_values(img, corners, dist, im_size)
    new_centers, new_avg_vals = merge_points(centers, centers_update, corners, avg_values, avg_values_corners, merge_threshold, distance_threshold)
        
    return new_centers, new_avg_vals

# update frame with new centers if avg value is greater than threshold
def update_frame(img, centers, dist, im_size, intensity_threshold, merge_threshold, distance_threshold):
    
    new_centers, new_avg_vals = get_center_values(img, centers, dist, im_size, merge_threshold, distance_threshold)
    
    for i in range(3):
        if new_avg_vals[i] > intensity_threshold:
            centers[i] = new_centers[i]
    
    return centers, new_avg_vals

# main function
def main(input_array, centers_ini, dist, im_size, intensity_threshold, merge_threshold, distance_threshold, num_iter):
    centers = centers_ini.copy()
    centers_array = []
    avg_values_array = []
    
    for i in range(num_iter):
        img = input_array[i].copy()
        centers_update, avg_values_temp = update_frame(img, centers, dist, im_size, intensity_threshold, merge_threshold, distance_threshold)
        centers_array.append(centers_update)
        avg_values_array.append(avg_values_temp)
        centers = centers_update.copy()
    
    return np.array(centers_array), np.array(avg_values_array)

In [ ]:
# initialize values
x_0 = np.array([1.33,-1.05]) # x-point location
l_0 = np.array([1.1,-1.3]) # left radiation point location
r_0 = np.array([1.42,-1.15]) # right radiation point location

centers_0 = np.array([convert_center(radii,elevation,x_0),
                      convert_center(radii,elevation,l_0),
                      convert_center(radii,elevation,r_0)])

x, y = zip(*centers_0)
img = inverted[0].copy()
plt.pcolormesh(radii[0],elevation[0],img,shading='auto', cmap='viridis')
plt.scatter(radii[0,x],elevation[0,y],s=5,c='tab:orange',label='intensity')
plt.xticks(np.arange(min(radii[0]), max(radii[0]), 0.1))
plt.yticks(np.arange(min(elevation[0]), max(elevation[0]), 0.1))
plt.show()

radius = 10 # for some reason, 6 just works the best while every other value doesn't work
threshold = 0.1 # intensity value threshold for points to update their position
merge_threshold = 1 # merge threshold for radiation point and X-point, should be on factor of 2X radius
distance_threshold = 1 # distance from emission line for corner to be considered a radiation point

# run main function
centers_array, avg_values_array = main(inverted, centers_0, radius, img.shape, threshold, merge_threshold, distance_threshold, len(inverted))

In [ ]:
# convert pixel location to natural units
r_natural = radii[0][centers_array[:,:,0]]
e_natural = elevation[0][centers_array[:,:,1]]
centers_array_natural = np.dstack((r_natural,e_natural))

### Results

In [ ]:
plt.plot(avg_values_array)
plt.legend(['x','l','r'])
plt.title(filename)
plt.show()

Circle bounds around point is a bit misleading, it's actually a square. But differences shouldn't be too big.

Intensity is the tracked points. Corners is any corners that gets noticed by detector.

In [ ]:
# sample plot of calculated centers
idx = 90
x, y = zip(*centers_array[idx])
img = inverted[idx].copy()

scaling = radii[0,1]-radii[0,0]

fig, axs = plt.subplots()
circ_0 = plt.Circle((radii[0,x[0]], elevation[0,y[0]]), scaling*radius, color='tab:red', fill=False)
circ_1 = plt.Circle((radii[0,x[1]], elevation[0,y[1]]), scaling*radius, color='tab:red', fill=False)
circ_2 = plt.Circle((radii[0,x[2]], elevation[0,y[2]]), scaling*radius, color='tab:red', fill=False)
line1x = [radii[0,x[0]],radii[0,x[1]]]
line1y = [elevation[0,y[0]],elevation[0,y[1]]]
line2x = [radii[0,x[0]],radii[0,x[2]]]
line2y = [elevation[0,y[0]],elevation[0,y[2]]]

axs.pcolormesh(radii[0],elevation[0],img,shading='auto', cmap='plasma')
axs.plot(line1x,line1y,color='tab:blue')
axs.plot(line2x,line2y,color='tab:blue')
axs.scatter(radii[0,x],elevation[0,y],s=5,c='tab:orange',label='intensity')
axs.add_artist(circ_0)
axs.add_artist(circ_1)
axs.add_artist(circ_2)

gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=useHarrisDetector))
x1 = radii[idx][corners[:,0,0]]
y1 = elevation[idx][corners[:,0,1]]
axs.scatter(x1,y1,color='cyan',s=5,marker='x',label='corners')
plt.suptitle(filename)
plt.title(f'Time = {times[idx]-times[0]:.2f} ms')
plt.legend()
plt.show()

In [ ]:
# generate video
savepath = Path('outputs/inversion_videos') / f'{filename}.gif'
fig, axs = plt.subplots()

def animate(idx):
    x, y = zip(*centers_array[idx])
    img = inverted[idx].copy()

    circ_0 = plt.Circle((radii[0,x[0]], elevation[0,y[0]]), scaling*radius, color='tab:red', fill=False)
    circ_1 = plt.Circle((radii[0,x[1]], elevation[0,y[1]]), scaling*radius, color='tab:red', fill=False)
    circ_2 = plt.Circle((radii[0,x[2]], elevation[0,y[2]]), scaling*radius, color='tab:red', fill=False)
    
    line1x = [radii[0,x[0]],radii[0,x[1]]]
    line1y = [elevation[0,y[0]],elevation[0,y[1]]]
    line2x = [radii[0,x[0]],radii[0,x[2]]]
    line2y = [elevation[0,y[0]],elevation[0,y[2]]]

    axs.clear()
    
    axs.pcolormesh(radii[0],elevation[0],img,shading='auto', cmap='plasma')
    axs.plot(line1x,line1y,color='tab:blue')
    axs.plot(line2x,line2y,color='tab:blue')
    axs.scatter(radii[0,x],elevation[0,y],s=5,c='tab:orange',label='intensity')
    axs.add_artist(circ_0)
    axs.add_artist(circ_1)
    axs.add_artist(circ_2)
    
    plt.xlabel('Radius (m)')
    plt.ylabel('Elevation (m)')
    plt.suptitle(filename)
    plt.title(f'Time = {times[idx]-times[0]:.2f} ms')

writervideo = animation.FFMpegWriter(fps=15) 
ani = animation.FuncAnimation(fig, animate, frames=tqdm(range(len(centers_array))))
ani.save(savepath, writer='Pillow', fps=15)
plt.close()

### Save Results

In [ ]:
datpath = Path('outputs/inversion_data')
dictionary = {'frame': frames,
              'x_location': centers_array[:,0,:],
              'l_location': centers_array[:,1,:],
              'r_location': centers_array[:,2,:],
              'x_intensity': avg_values_array[:,0],
              'l_intensity': avg_values_array[:,1],
              'r_intensity': avg_values_array[:,2]}
    
savepkl = (datpath / filename).with_suffix('.pkl')

with open(savepkl, 'wb') as f:
    pickle.dump(dictionary, f)

### Iterate Across All Files

In [ ]:
file_name = [f for f in filepath.glob('*') if f.is_file()]
print(file_name[13:])

In [ ]:
filepath = Path('tv_images')
datpath = Path('inversion_data')
vidpath = Path('inversion_videos')

file_name = [f for f in filepath.glob('*') if f.is_file()]
[inverted,radii,elevation,frames,_,_,_,_] = _load_data(file_name[0])
# initialize values
x_0 = np.array([1.35,-1.05]) # x-point location
l_0 = np.array([1.0,-1.25]) # left radiation point location
r_0 = np.array([1.5,-1.25]) # right radiation point location

centers_0 = np.array([convert_center(radii,elevation,x_0),
                      convert_center(radii,elevation,l_0),
                      convert_center(radii,elevation,r_0)])

radius = 6 # for some reason, 6 just works the best while every other value doesn't work
threshold = 0.05 # intensity value threshold for points to update their position
merge_threshold = 20 # merge threshold for radiation point and X-point, should be on factor of 2X radius
distance_threshold = 10 # distance from emission line for corner to be considered a radiation point

for file in tqdm(file_name[14:]):
    print(file.stem)
    [inverted,radii,elevation,frames,times,_,_,_] = _load_data(file)
    r = radii[0]
    z = elevation[0]
    centers_array, avg_values_array = main(inverted, centers_0, radius, inverted[0].shape, threshold, merge_threshold, distance_threshold, len(inverted))
    centers_array_natural = np.dstack((radii[0][centers_array[:,:,0]],elevation[0][centers_array[:,:,1]]))
    dictionary = {'frame': frames,
              'x_location': centers_array_natural[:,0,:],
              'l_location': centers_array_natural[:,1,:],
              'r_location': centers_array_natural[:,2,:],
              'x_intensity': avg_values_array[:,0],
              'l_intensity': avg_values_array[:,1],
              'r_intensity': avg_values_array[:,2]}
    
    savepkl = (datpath / file.stem).with_suffix('.pkl')
    
    with open(savepkl, 'wb') as f:
        pickle.dump(dictionary, f)
        
    savevid = (vidpath / file.stem).with_suffix('.mp4')
    
    fig, axs = plt.subplots()

    def animate(idx):
        x, y = zip(*centers_array[idx])
        img = inverted[idx].copy()

        circ_0 = plt.Circle((radii[0,x[0]], elevation[0,y[0]]), scaling*radius, color='tab:red', fill=False)
        circ_1 = plt.Circle((radii[0,x[1]], elevation[0,y[1]]), scaling*radius, color='tab:red', fill=False)
        circ_2 = plt.Circle((radii[0,x[2]], elevation[0,y[2]]), scaling*radius, color='tab:red', fill=False)
        
        line1x = [radii[0,x[0]],radii[0,x[1]]]
        line1y = [elevation[0,y[0]],elevation[0,y[1]]]
        line2x = [radii[0,x[0]],radii[0,x[2]]]
        line2y = [elevation[0,y[0]],elevation[0,y[2]]]

        axs.clear()
        
        axs.pcolormesh(radii[0],elevation[0],img,shading='auto', cmap='plasma')
        axs.plot(line1x,line1y,color='tab:blue')
        axs.plot(line2x,line2y,color='tab:blue')
        axs.scatter(radii[0,x],elevation[0,y],s=5,c='tab:orange',label='intensity')
        axs.add_artist(circ_0)
        axs.add_artist(circ_1)
        axs.add_artist(circ_2)
        
        plt.xlabel('Radius (m)')
        plt.ylabel('Elevation (m)')
        plt.suptitle(file.stem)
        plt.title(f'Time = {times[idx]-times[0]:.2f} ms')

    writervideo = animation.FFMpegWriter(fps=15) 
    ani = animation.FuncAnimation(fig, animate, frames=tqdm(range(len(centers_array)-1)))
    ani.save(savevid, writer=writervideo)
    plt.close()

### Semi-Supervised Identification

In [ ]:
filepath = Path('tv_images')
datpath = Path('inversion_data')
modelpath = Path('models')

file_name = 'lr_inversion_points.pkl'

with open(modelpath / file_name, 'rb') as f:
    inversion_model = pickle.load(f)

file_name = [f for f in filepath.glob('*') if f.is_file()]
file = file_name[0]
inversion_vid = readsav(file)['emission_structure'][0][0]
inversion_vid_copy = inversion_vid.copy()
inversion_vid = (255-255*(inversion_vid-np.min(inversion_vid))/(np.max(inversion_vid)-np.min(inversion_vid)))/255
inversion_vid = inversion_vid.reshape((len(inversion_vid), -1))


In [ ]:
with open((datpath / file_name[0].stem).with_suffix('.pkl'), 'rb') as f:
    inversion_data = pickle.load(f)
    
l = inversion_data['l_location']
r = inversion_data['r_location']
points_arr = np.concatenate((l, r),1)

In [ ]:
predict = inversion_model.predict(inversion_vid)

In [ ]:
score = inversion_model.score(inversion_vid, points_arr)
print(score)

In [ ]:
fig, ax = plt.subplots()

# Create scatter plots
bg_vid = ax.imshow(inversion_vid_copy[0], cmap='gray', origin='lower', extent=[1,2,-1.4,-.4])
scat_pred = ax.scatter([], [], c='lime', label='predicted')
scat_actual = ax.scatter([], [], c='red', label='actual')
ax.set_xlim([1,2])
ax.set_ylim([-1.4,-.4])
ax.legend()
ax.set_title("Emission Front Locations")
plt.tight_layout()

def update(num):
    x1, y1, x2, y2 = predict[num]
    a1, b1, a2, b2 = points_arr[num]
    bg_vid.set_data(inversion_vid_copy[num])
    
    scat_pred.set_offsets(np.c_[[x1, x2], [y1, y2]])
    scat_actual.set_offsets(np.c_[[a1, a2], [b1, b2]])
    return scat_pred, scat_actual, bg_vid

FFwriter = animation.FFMpegWriter(fps=60)
ani = animation.FuncAnimation(fig, update, frames=tqdm(range(len(predict))), interval=20, blit=True)
ani.save(Path(f'./tmp/{file.stem}_regress2.mp4'), writer=FFwriter)
plt.close()

In [ ]:
def _find_index(arr,val):
    return np.argmin(abs(arr-val))

def get_index(radius_map, elevation_map, point_array):
    rad_coord = list(map(_find_index, radius_map, point_array[0]))
    ele_coord = list(map(_find_index, elevation_map, point_array[1]))
    return np.array([rad_coord, ele_coord]).T

rad_map = readsav(file)['emission_structure'][0][1]
ele_map = readsav(file)['emission_structure'][0][2]
l_point = predict.T[[0,1],:]
r_point = predict.T[[2,3],:]
l_point_coord = get_index(rad_map, ele_map, l_point)
r_point_coord = get_index(rad_map, ele_map, r_point)

In [ ]:
frame_size = 13
l_frame = np.array([l_point_coord[:,0]-frame_size,l_point_coord[:,0]+frame_size,
                    l_point_coord[:,1]-frame_size,l_point_coord[:,1]+frame_size]).T
l_frame = np.where(l_frame < 0, 0, l_frame)
l_frame = np.where(l_frame > 201, 201, l_frame)

r_frame = np.array([r_point_coord[:,0]-frame_size,r_point_coord[:,0]+frame_size,
                    r_point_coord[:,1]-frame_size,r_point_coord[:,1]+frame_size]).T
r_frame = np.where(r_frame < 0, 0, r_frame)
r_frame = np.where(r_frame > 201, 201, r_frame)

In [ ]:
def crop_vid(vid, frame, idx):
    return vid[idx][frame[idx,2]:frame[idx,3],frame[idx,0]:frame[idx,1]]

In [ ]:
cropped_vid = [crop_vid(inversion_vid_copy, l_frame, i) for i in range(len(inversion_vid_copy))]
max_loc = np.array([np.flip(np.unravel_index(np.argmax(cropped_vid[i]), cropped_vid[i].shape)) for i in range(len(cropped_vid))])
true_max_loc_l = max_loc + l_frame[:,[0,2]]

In [ ]:
cropped_vid = [crop_vid(inversion_vid_copy, r_frame, i) for i in range(len(inversion_vid_copy))]
max_loc = np.array([np.flip(np.unravel_index(np.argmax(cropped_vid[i]), cropped_vid[i].shape)) for i in range(len(cropped_vid))])
true_max_loc_r = max_loc + r_frame[:,[0,2]]

In [ ]:
plt.imshow(cropped_vid[5], origin='lower')

In [ ]:
idx = 120

plt.imshow(inversion_vid_copy[idx], cmap = 'grey', origin = 'lower')
plt.scatter(true_max_loc_l[idx][0], true_max_loc_l[idx][1], c='red', label = 'adjusted',s=5)
plt.scatter(l_point_coord[idx][0], l_point_coord[idx][1], c='cyan', label = 'original',s=3)
plt.scatter(true_max_loc_r[idx][0], true_max_loc_r[idx][1], c='red',s=5)
plt.scatter(r_point_coord[idx][0], r_point_coord[idx][1], c='cyan',s=3)
rect = patches.Rectangle((l_frame[idx,0],l_frame[idx,2]),l_frame[idx,1]-l_frame[idx,0],l_frame[idx,3]-l_frame[idx,2],linewidth=1,edgecolor='r',facecolor='none')
rect2 = patches.Rectangle((r_frame[idx,0],r_frame[idx,2]),r_frame[idx,1]-r_frame[idx,0],r_frame[idx,3]-r_frame[idx,2],linewidth=1,edgecolor='r',facecolor='none')
plt.gca().add_patch(rect)
plt.gca().add_patch(rect2)
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()

# Create scatter plots
bg_vid = ax.imshow(inversion_vid_copy[0], cmap='gray', origin='lower')
scat_l = ax.scatter([], [], c='cyan', label='regression', s=10)
scat_r = ax.scatter([], [], c='cyan', s=10)
scat_l_adj = ax.scatter([], [], c='red', label='adjusted', s=5)
scat_r_adj = ax.scatter([], [], c='red', s=5)
rect_p = ax.add_patch(patches.Rectangle((l_frame[0,0],l_frame[0,2]),l_frame[0,1]-l_frame[0,0],l_frame[0,3]-l_frame[0,2],linewidth=1,edgecolor='r',facecolor='none'))
rect2_p = ax.add_patch(patches.Rectangle((r_frame[0,0],r_frame[0,2]),r_frame[0,1]-r_frame[0,0],r_frame[0,3]-r_frame[0,2],linewidth=1,edgecolor='r',facecolor='none'))
ax.legend()
ax.set_title("Emission Front Locations")
plt.tight_layout()

def update(num):
    bg_vid.set_data(inversion_vid_copy[num])
    scat_l.set_offsets(l_point_coord[num])
    scat_r.set_offsets(r_point_coord[num])
    scat_l_adj.set_offsets(true_max_loc_l[num])
    scat_r_adj.set_offsets(true_max_loc_r[num])
    rect = patches.Rectangle((l_frame[num,0],l_frame[num,2]),l_frame[num,1]-l_frame[num,0],l_frame[num,3]-l_frame[num,2],linewidth=1,edgecolor='r',facecolor='none')
    rect2 = patches.Rectangle((r_frame[num,0],r_frame[num,2]),r_frame[num,1]-r_frame[num,0],r_frame[num,3]-r_frame[num,2],linewidth=1,edgecolor='r',facecolor='none')
    rect_p.set_xy((l_frame[num,0],l_frame[num,2]))
    rect2_p.set_xy((r_frame[num,0],r_frame[num,2]))
    return bg_vid, scat_l, scat_r, scat_l_adj, scat_r_adj, rect_p, rect2_p

FFwriter = animation.FFMpegWriter(fps=30)
ani = animation.FuncAnimation(fig, update, frames=tqdm(range(len(predict))), interval=20, blit=True)
ani.save(Path(f'./tmp/{file.stem}_inversion_vid.gif'), writer='Pillow')
plt.close()
